In [92]:
import pandas as pd
import os
import sys
import yaml
import numpy as np

In [93]:
instruments = pd.read_csv('../../Average_holdings_2015.csv')
mean_holding_period = round(instruments.days.mean())
print('mean holding period:', mean_holding_period, 'days')

mean holding period: 25 days


In [94]:
inst_dict = dict(zip(instruments.Symbol, instruments.days))

In [95]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
start_date = '2013-06-03'
end_date = '2015-12-31'

In [96]:
raw_data_path = 'raw/'
original_path = 'ourpped/'
testing_oris_method = 'testing_oris_method/'

stocknet_tickers = ['XOM', 'RDS-B', 'PTR', 'CVX', 'TOT', 'BP', 'BHP', 'SNP', 'SLB', 'BBL',
                    'AAPL', 'PG', 'BUD', 'KO', 'PM', 'TM', 'PEP', 'UN', 'UL', 'MO',
                    'JNJ', 'PFE', 'NVS', 'UNH', 'MRK', 'AMGN', 'MDT', 'ABBV', 'SNY', 'CELG',
                    'AMZN', 'BABA', 'WMT', 'CMCSA', 'HD', 'DIS', 'MCD', 'CHTR', 'UPS', 'PCLN',
                    'NEE', 'DUK', 'D', 'SO', 'NGG', 'AEP', 'PCG', 'EXC', 'SRE', 'PPL',
                    'IEP', 'HRG', 'CODI', 'REX', 'SPLP', 'PICO', 'AGFS', 
                    'BCH', 'BSAC', 'BRK-A', 'JPM', 'WFC', 'BAC', 'V', 'C', 'HSBC', 'MA',
                    'GE', 'MMM', 'BA', 'HON', 'UTX', 'LMT', 'CAT', 'GD', 'DHR', 'ABB',
                    'GOOG', 'MSFT', 'FB', 'T', 'CHL', 'ORCL', 'TSM', 'VZ', 'INTC', 'CSCO' ] #'GMRE'

print('num of intersection between lists:', len(set(inst_dict.keys()).intersection(stocknet_tickers)))

### create dict for mean holdings for stocknet
stocknet_tickers_dict = {}
for s in stocknet_tickers:
    try:
        stocknet_tickers_dict[s] = int(Inst_dict[s])
    except:
        stocknet_tickers_dict[s] = int(mean_holding_period)


num of intersection between lists: 52


In [97]:
mult_percentege = 100 #adjust according to pecentage on 'ourpped'
def up_days(s):    
    stock_movement_path = os.path.join(raw_data_path, '{}.csv'.format(s))
    df = pd.read_csv(stock_movement_path)
    
    #append mean holdings per stock
    mean_holding = stocknet_tickers_dict[s]

    #### Adjusting accordingly to the stocknet ####

    # vector that normalize the prices by adjusted_price(t-1)
    norm_vector = pd.Series(df['Adj Close']).shift(1).fillna(method ='bfill') #fill the first value with the second

#     # norm by close at t
#     col_adj = ['Open', 'High', 'Low', 'Volume']
#     for col in col_adj:
#         stock_sample[col] = stock_sample[col]/stock_sample['Close'] -1

    ## add my own features ##
    #1) avergae mean per each stock
    df.loc[:,'average_mean'] = df.loc[:,'Adj Close'].rolling(mean_holding).mean()
    df.loc[:,'average_mean'] = mult_percentege*(df.loc[:,'average_mean']/df.loc[:,'Adj Close'] -1) #adjust according to paper
    
    #norm by 'close' at t-1
    col_adj_t_minus_1 = ['Close', 'Adj Close']
    for col in col_adj_t_minus_1:
        df[col] = df[col]/norm_vector -1
    
    #2) up most of the days
    df.loc[:,'up_days_mean']=\
    df.loc[:,'Adj Close'].rolling(mean_holding).agg(lambda x: (x > 0).mean())
    df.loc[:,'up_days_mean'] = MinMaxScaler().fit_transform(df.loc[:,'up_days_mean'].values.reshape(-1, 1))
    
    #mask between dates
    mask = (df['Date'] >= start_date) & (df['Date'] <= end_date)
    df = df.loc[mask]
    
    return  np.vstack([df['average_mean'].values, df['up_days_mean'].values]).T  #df['up_days_mean'].values, df['average_mean'].values #


In [98]:
### Insert new column to the orig df
def change_orig_df(s, num_row=0):
    #original df
    df_orig = pd.read_csv(os.path.join(original_path,'{}.csv'.format(s)),header=None)
    df_new = df_orig.copy()

    #insert new variable - up days
    if num_row == 0:
        tmp_last_columns = df_new.iloc[:,-2:].copy()
        df_new.iloc[:,-2:]  = up_days(s)   # was  [:, -3], [:,-4:-2]
        df_new = pd.concat([df_new, pd.DataFrame(tmp_last_columns)], axis=1)
    else:
        tmp_last_columns = df_new.iloc[-num_row:,-2:].copy()
        df_new.iloc[-num_row:,-2:]  = up_days(s)[-num_row:]
        df_new = pd.concat([df_new, pd.DataFrame(tmp_last_columns)], axis=1)
        
        #df_new.iloc[-num_row:, -3] = up_days(s)[-num_row:]      # was [:,-3], up_days(s)[-num_row:]  df_new.iloc[-num_row:, -4:-2]
    return df_new

In [99]:
for s in stocknet_tickers:
    try:
        df_new = change_orig_df(s).round(6)
        df_new.to_csv(os.path.join(testing_oris_method,'{}_testing_avgholdings2015.csv'.format(s)),header=None, index=None)
    except:
        print('except:', s)


# df_new = change_orig_df(s).round(6)
# df_new.to_csv(os.path.join(testing_oris_method,'{}_testing_avgholdings2015.csv'.format(s)),header=None, index=None)

except: PTR
except: TOT
except: SNP
except: SLB
except: BBL
except: UN
except: UL
except: NVS
except: SNY
except: BABA
except: PCLN
except: NGG
except: HRG
except: CODI
except: REX
except: SPLP
except: PICO
except: AGFS
except: BRK-A
except: ABB
except: TSM


### Implement for all stocks

In [13]:
raw_data_path = 'raw/'
original_path = 'ourpped/'
s1 = 'AAPL'
s2 = 'AMZN'
stock_path = os.path.join(raw_data_path, '{}.csv'.format(s1))
stock_sample = pd.read_csv(stock_path)

meanholding_tmp = int(inst_dict[s1])
stock_sample.loc[:, 'average_mean'] = stock_sample.loc[:, 'Adj Close'].rolling(meanholding_tmp).mean()
#norm by adj close
mult_percentage = 100
stock_sample.loc[:, 'average_mean'] = mult_percentage*(stock_sample.loc[:, 'average_mean']/
                                                       stock_sample.loc[:, 'Adj Close'] - 1)

# adjusting accordingly to the txt #
# norm by close at t
norm_vector = pd.Series(stock_sample['Adj Close']).shift(1).fillna(method='bfill')
col_adj = ['Open', 'High', 'Low', 'Volume']
for col in col_adj:
    stock_sample[col] = stock_sample[col]/stock_sample['Close'] - 1
    
# norm by close at t-1
col_adj_t_minus_1 = ['Close', 'Adj Close']
for col in col_adj_t_minus_1:
    stock_sample[col] = stock_sample[col]/norm_vector - 1
    
df = stock_sample.copy()

In [16]:
df.loc[:, 'up_days_mean'] = df.loc[:, 'Adj Close'].rolling(meanholding_tmp).agg(lambda x: (x>0).mean())
#scale
df.loc[:, 'up_days_mean'] = MinMaxScaler().fit_transform(df.loc[:, 'up_days_mean'].values.reshape(-1, 1))

#change order
new_df_order = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'average_mean', 'up_days_mean', 'Volume']
df = df[new_df_order]
df.tail(1)

,Date,Open,High,Low,Close,Adj Close,average_mean,up_days_mean,Volume
1257,2017-09-01,0.004572,0.005425,-0.00256,0.000305,0.000305,-8.527576,0.782609,100899.942989


In [17]:
mask = (df['Date'] >= start_date) & (df['Date'] <= end_date)
df = df.loc[mask]
df.drop('Date', axis=1, inplace=True)

In [21]:
original_path = 'ourpped/'
df_orig = pd.read_csv(os.path.join(original_path,'{}.csv'.format(s1)),header=None)
df_orig.head()
df_new = df_orig.copy()

In [25]:
tmp_new_columns = np.vstack([df['average_mean'].values, df['up_days_mean'].values]).T
tmp_last_columns = df_new.iloc[:, -2:].copy()
df_new.iloc[:, -2:] = tmp_new_columns
df_new = pd.concat([df_new, pd.DataFrame(tmp_last_columns)], axis=1)

In [26]:
df_new.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,11,12
0,0.002215,0.363858,-1.828186,0.220128,0.220127,-0.673593,-1.276852,-1.713707,-0.949655,-1.295995,-2.789459,-0.580885,0.347826,0.0,59.171677
1,0.870223,1.139527,-0.427319,-0.312838,-0.312846,-0.011565,-0.825038,-1.485826,-0.731958,-0.790216,-2.089251,-0.453149,0.347826,0.0,58.986561
2,0.121322,1.260365,-0.314531,-0.934764,-0.934782,0.939114,0.233220,-0.537524,0.086406,0.193033,-0.853451,0.323316,0.347826,-1.0,58.435165
3,1.598781,1.947728,-1.005795,-1.494014,-1.493998,1.871554,1.687500,1.117097,1.349731,1.731397,0.921408,1.671675,0.347826,-1.0,57.562145
4,-1.201875,0.323667,-2.046127,0.764040,0.764051,0.740579,0.908979,0.459466,0.411936,0.952945,0.428645,0.733115,0.347826,1.0,58.001949


In [27]:
testing_oris_method = 'testing_oris_method/'
df_new.to_csv(os.path.join(testing_oris_method, '{}.csv'.format(s1)), header=None, index=None)